In [1]:
import sympy as sp
import roboticstoolkit as rtk

# Rotations and Transformations

### Describing Transformations

> Consider 3 frames:  
> Frame 1 is stationary, and frames 2 and 3 are initially coincident with frame 1.  
> Frame 2 is rotated about the $x_2$ axis by $-28^\circ$, then about $y_2$ by $-79^\circ$ and then about $z_2$ by $56^\circ$.  
> Frame 3 is rotated about the $z_1$ axis by $28^\circ$, then about $y_1$ by $-79^\circ$ and then about $x_1$ by $56^\circ$. Finally, the origin of frame 3 is translated to $[1, 2, 0]^T$.
>
> Find the rotation matrix that maps a point from frame 2 to frame 1. Given a point ${}^1P=[4, -2, 2]^T$, find ${}^2P$
> Find the transformation matrices ${}^1_3T$ and ${}^3_1T$

Frame 2 is rotated about the axes of the moving frame, so these are Euler angles. Postmultiply rotation matrices to give a rotation that maps a point from frame 2 to frame 1

In [2]:
rotation = rtk.rot_x(-28 * sp.pi/180) * rtk.rot_y(-79 * sp.pi/180) * rtk.rot_z(56 * sp.pi/180)
rotation = rtk.rotation(sp.N(sp.simplify(rotation), 3))
rotation

Matrix([
[ 0.107, -0.158, -0.982],
[  0.99,  0.112, 0.0896],
[0.0955, -0.981,  0.168]])

Find ${}^2P$:

In [3]:
p_in_1 = sp.Matrix([4, -2, 2])
rotation.T * p_in_1

Matrix([
[-1.36],
[-2.82],
[-3.77]])

Frame 3 is rotated about the axes of the fixed frame, so these are fixed angles. Premultiply rotation matrices to give a rotation that maps a point from frame 1 to a frame parallel to frame 3

In [4]:
rotation = rtk.rot_z(28 * sp.pi/180) * rtk.rot_y(-79 * sp.pi/180) * rtk.rot_x(56 * sp.pi/180)
rotation = rtk.rotation(sp.N(sp.simplify(rotation), 3))
rotation

Matrix([
[ 0.168, -0.981, -0.0955],
[0.0896,  0.112,   -0.99],
[ 0.982,  0.158,   0.107]])

Find the transformation matrices:

In [5]:
t_3_in_1 = rtk.trans(1, 2, 0)
t_3_in_1[:3, :3] = rotation.T
t_3_in_1

Matrix([
[  0.168, 0.0896, 0.982, 1],
[ -0.981,  0.112, 0.158, 2],
[-0.0955,  -0.99, 0.107, 0],
[      0,      0,     0, 1]])

In [6]:
t_1_in_3 = rtk.inverse_transform(t_3_in_1)
t_1_in_3

Matrix([
[ 0.168, -0.981, -0.0955,   1.79],
[0.0896,  0.112,   -0.99, -0.313],
[ 0.982,  0.158,   0.107,   -1.3],
[     0,      0,       0,      1]])

Check using the identity matrix:

In [7]:
t_1_in_3 * t_3_in_1

Matrix([
[    1.0,   0, 7.63e-6, 0],
[      0, 1.0,       0, 0],
[7.63e-6,   0,     1.0, 0],
[      0,   0,       0, 1]])

### Combining Transformations

>In a 6-DOF robotic system, a camera is attached to the fifth link of the robot.
The sixth link is the end-effector. The camera observes an object and determines its frame
relative to the camera’s frame. Four frames, $F_5$, $F_E$, $F_{cam}$ and $F_{obj}$ , are attached to the
fifth link, the end-effector, the camera, and the object, respectively.
>
>$$
{}^5T_{cam} = \left[\begin{matrix}
0 & 0 & -1 & 2\\
0 & -1 & 0 & 1\\
-1 & 0 & 0 & 6\\
0 & 0 & 0 & 1
\end{matrix}\right],\quad
\
{}^5T_E = \left[\begin{matrix}
0 & -1 & 0 & 0\\
1 & 0 & 0 & 2\\
0 & 0 & 1 & 5\\
0 & 0 & 0 & 1
\end{matrix}\right],\quad
\
{}^{cam}T_{obj} = \left[\begin{matrix}
0 & 0 & 1 & 2\\
1 & 0 & 0 & 3\\
0 & 1 & 0 & 5\\
0 & 0 & 0 & 1
\end{matrix}\right]$$
>
> Using the above information, determine the transformation matrix between $F_E$ and $F_{obj}$ and the distance between the end-effector and the object.

Find transformation matrix:

In [8]:
t_cam_in_5 = sp.Matrix([
    [0, 0, -1, 2],
    [0, -1, 0, 1],
    [-1, 0, 0, 6],
    [0, 0, 0, 1]
])
t_e_in_5 = sp.Matrix([
    [0, -1, 0, 0],
    [1, 0, 0, 2],
    [0, 0, 1, 5],
    [0, 0, 0, 1]
])
t_obj_in_cam = sp.Matrix([
    [0, 0, 1, 2],
    [1, 0, 0, 3],
    [0, 1, 0, 5],
    [0, 0, 0, 1]
])
t_obj_in_e = rtk.inverse_transform(t_e_in_5) * t_cam_in_5 * t_obj_in_cam
t_obj_in_e

Matrix([
[-1, 0,  0, -4],
[ 0, 1,  0,  3],
[ 0, 0, -1, -1],
[ 0, 0,  0,  1]])

Distance between end effector and object is the distance between the origins of the frames:

In [9]:
p_obj_in_e = rtk.translation(t_obj_in_e)
p_obj_in_e

Matrix([
[-4],
[ 3],
[-1]])

In [10]:
sp.sqrt(p_obj_in_e.dot(p_obj_in_e))

sqrt(26)

# Forward Kinematics

### DH Parameters

> Given a manipulator with the following DH parameters, find the incremental transformation matrices.  
> Also find the matrix ${}^0_4T$ with the values $\theta_1 = 90^\circ$, $\theta_2 = 45^\circ$, $\theta_3 = -45^\circ$, $\theta_4 = 90^\circ$, $L_1 = L_2 = L_3 = L_4  =1$
>
>|   | $a_{i-1}$   | $\alpha_{i-1}$ | $d_i$   | $\theta_i$   |
>|---|-------------|---------------|----------|--------------|
>| 1 | $0$         | $0$           | $L_1$    | $\theta_1$   |
>| 2 | $L_2$       | $-\pi/2$      | $0$      | $\theta_2$   |
>| 3 | $0$         | $\pi/2$       | $L_3$    | $\theta_3$   |
>| 4 | $L_4$       | $-\pi/2$      | $0$      | $\theta_4$   |


In [11]:
L1, L2, L3, L4, theta1, theta2, theta3, theta4 = sp.symbols('L1, L2, L3, L4, theta1, theta2, theta3, theta4')
dh_table = [
    [0, 0, L1, theta1],
    [L2, -sp.pi/2, 0, theta2],
    [0, sp.pi/2, L3, theta3],
    [L4, -sp.pi/2, 0, theta4]
]
link_transforms = rtk.link_transforms(dh_table)
link_transforms

[Matrix([
 [cos(theta1), -sin(theta1), 0,  0],
 [sin(theta1),  cos(theta1), 0,  0],
 [          0,            0, 1, L1],
 [          0,            0, 0,  1]]),
 Matrix([
 [ cos(theta2), -sin(theta2), 0, L2],
 [           0,            0, 1,  0],
 [-sin(theta2), -cos(theta2), 0,  0],
 [           0,            0, 0,  1]]),
 Matrix([
 [cos(theta3), -sin(theta3),  0,   0],
 [          0,            0, -1, -L3],
 [sin(theta3),  cos(theta3),  0,   0],
 [          0,            0,  0,   1]]),
 Matrix([
 [ cos(theta4), -sin(theta4), 0, L4],
 [           0,            0, 1,  0],
 [-sin(theta4), -cos(theta4), 0,  0],
 [           0,            0, 0,  1]])]

In [12]:
subs_map = {theta1:sp.pi/2, theta2:sp.pi/4, theta3:-sp.pi/4, theta4:sp.pi/2, L1:1, L2:1, L3:1, L4:1}
t_4 = rtk.end_transform(dh_table).subs(subs_map)
t_4

Matrix([
[         0, -sqrt(2)/2, -sqrt(2)/2,       sqrt(2)/2],
[-sqrt(2)/2,       -1/2,        1/2, sqrt(2)/2 + 3/2],
[-sqrt(2)/2,        1/2,       -1/2, 1/2 + sqrt(2)/2],
[         0,          0,          0,               1]])

In [13]:
sp.N(t_4, 4)

Matrix([
[      0, -0.7071, -0.7071, 0.7071],
[-0.7071,    -0.5,     0.5,  2.207],
[-0.7071,     0.5,    -0.5,  1.207],
[      0,       0,       0,    1.0]])

### Forward Kinematics

> The transformation matrices and end effector position vector for a robot are given by
>$$
{}^0T_1 = \left[\begin{matrix}
c_1 & s_1 & 0 & 0\\
-s_1 & c_1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1
\end{matrix}\right],\quad
\
{}^1T_2 = \left[\begin{matrix}
0 & -1 & 0 & 0\\
0 & 0 & -1 & -d_2\\
1 & 0 & 0 & 0\\
0 & 0 & 0 & 1
\end{matrix}\right],\quad
\
{}^2T_3 = \left[\begin{matrix}
c_3 & -s_3 & 0 & 0\\
0 & 0 & 1 & 0\\
-s_3 & -c_3 & 0 & 0\\
0 & 0 & 0 & 1
\end{matrix}\right], \quad
\
{}^3p_{EE} = \left[\begin{matrix}
L\\
0\\
0
\end{matrix}\right]$$
> where $c_i = \cos(\theta_i)$ and $s_i = \sin(\theta_i)$
>
> Given the joint variables $q = [\theta_1, d_2, \theta_3]^T$, derive the forward kinematics solutions for this robot giving a general position of the end effector as below.
>$${}^0p_{EE} = \left[\begin{matrix}
x\\
y\\
z
\end{matrix}\right]$$

Calculate the forward kinematics:

In [14]:
theta1, d2, theta3, L = sp.symbols('theta1, d2, theta3, L')
t_1 = rtk.rot_z(-theta1)
t_2 = sp.Matrix([
    [0, -1, 0, 0, ],
    [0, 0, -1, -d2],
    [1, 0, 0, 0],
    [0, 0, 0, 1]
])
t_3 = sp.Matrix([
    [sp.cos(theta3), -sp.sin(theta3), 0, 0],
    [0, 0, 1, 0],
    [-sp.sin(theta3), -sp.cos(theta3), 0, 0],
    [0, 0, 0, 1]
])
p_ee = rtk.four_vector(sp.Matrix([L, 0, 0]))
dk = t_1 * t_2 * t_3 * p_ee
dk

Matrix([
[L*sin(theta1)*sin(theta3) - d2*sin(theta1)],
[L*sin(theta3)*cos(theta1) - d2*cos(theta1)],
[                             L*cos(theta3)],
[                                         1]])

In [15]:
rtk.three_vector(dk)

Matrix([
[L*sin(theta1)*sin(theta3) - d2*sin(theta1)],
[L*sin(theta3)*cos(theta1) - d2*cos(theta1)],
[                             L*cos(theta3)]])

We can use this to check an inverse kinematics solution.
Consider the following end effector position, using $L=12$ and $d_2 > 10$:
$${}^0p_{EE} = \left[\begin{matrix}
x\\
y\\
z
\end{matrix}\right] = \left[\begin{matrix}
5\\
8\\
7
\end{matrix}\right]$$

Analytically, the inverse kinematics is given below:
$$\theta_3 = \pm\arccos(\frac{z}{L})$$
$$d_2 = L\sin(\theta_3) \pm \sqrt{x^2 + y^2}$$
$$\theta_1 = \arctan2(\frac{x}{L\sin(\theta_3) - d_2}, \frac{y}{L\sin(\theta_3) - d_2})$$

In [16]:
x_val, y_val, z_val = 5, 8, 7
L_val = 12
theta3_val = sp.acos(z_val/L_val)
sp.N(theta3_val * 180/sp.pi, 4)

54.31

In [17]:
d_term1 = L_val*sp.sin(theta3_val)
d_term2 = sp.sqrt(x_val**2 + y_val**2)
print(sp.N(d_term1, 4))
print(sp.N(d_term2, 4))

9.747
9.434


We want $d_2 > 10$, so require both terms for $d_2$ be positive. This also requires $\theta_3 > 0$.

In [18]:
d2_val = d_term1 + d_term2
sp.N(d2_val, 4)

19.18

In [19]:
denom = L_val * sp.sin(theta3_val) - d2_val
theta1_val = sp.atan2(x_val/denom, y_val/denom)
sp.N(theta1_val * 180/sp.pi, 4)

-148.0

Substitute the values for the joint variables:
$$\theta_1 = -148.0^\circ$$
$$d_2 = 19.18$$
$$\theta_3 = 54.31^\circ$$

In [20]:
sp.N(dk.subs({
    theta1:theta1_val,
    d2:d2_val,
    theta3:theta3_val,
    L:L_val
}),4)

Matrix([
[5.0],
[8.0],
[7.0],
[1.0]])

# Dynamics

### Lagrange Dynamics

> Derive the kinetic (T) and potential energies (V) for each link of a robot where the information 
is given below. Assume that the links are modelled as point masses, $m_1$, $m_2$ and $m_3$. Each 
point mass is located at the middle of each link or at the end effector. There is no external 
force or moment on the end effector.
>
> The actuated joints for this robot are $q = [\theta_1, \theta_2, d_3]$.
>
> Transformation matrices:
>$$
{}^0T_1 = \left[\begin{matrix}
c_1 & -s_1 & 0 & 0\\
s_1 & c_1 & 0 & 0\\
0 & 0 & 1 & L_1\\
0 & 0 & 0 & 1
\end{matrix}\right],\quad
\
{}^1T_2 = \left[\begin{matrix}
c_2 & -s_2 & 0 & 0\\
0 & 0 & -1 & 0\\
s_2 & c_2 & 0 & 0\\
0 & 0 & 0 & 1
\end{matrix}\right],\quad
\
{}^2T_3 = \left[\begin{matrix}
1 & 0 & 0 & 0\\
0 & 0 & 1 & d_3\\
0 & -1 & 0 & 0\\
0 & 0 & 0 & 1
\end{matrix}\right]$$
> Locations of mass centres:
>$$
{}^1p_{c1} = \left[\begin{matrix}
0\\
0\\
-L_1/2
\end{matrix}\right],\quad
\
{}^2p_{c2} = \left[\begin{matrix}
0\\
d_3/2\\
0
\end{matrix}\right],\quad
\
{}^3p_{c3} = \left[\begin{matrix}
0\\
0\\
0
\end{matrix}\right]$$
> Gravity is acting in the negative $z_0$ direction
>
> Also find the numerical value of the Lagrangian with the following variables:
>$$L_1 = 0.5\ m$$
>$$q = [28^\circ \quad 79^\circ \quad 0.225\ m]^T$$
>$$\dot{q} = [0.12\ rad/s \quad 0.05\ rad/s \quad 0.56\ m/s]^T$$
>$$g=10\ m/s^2$$
>$$m_1 = 2.8\ kg, \quad m_2 = 1.4\ kg, \quad m_3 = 0.8\ kg$$

In [21]:
# Define symbolic variables
theta1, theta2, d3 = sp.symbols('theta1, theta2, d3')
L1, m1, m2, m3, g = sp.symbols('L1, m1, m2, m3, g')

# Define the manipulator
transforms = [
    rtk.screw_z(L1, theta1),
    sp.Matrix([
        [sp.cos(theta2), -sp.sin(theta2), 0, 0],
        [0, 0, -1, 0],
        [sp.sin(theta2), sp.cos(theta2), 0, 0],
        [0, 0, 0, 1]
    ]),
    sp.Matrix([
        [1, 0, 0, 0],
        [0, 0, 1, d3],
        [0, -1, 0, 0],
        [0, 0, 0, 1]
    ]),
    rtk.trans(0, 0, 0)
]
pos_coms = [
    sp.zeros(3, 1),
    sp.Matrix([0, 0, -L1/2]),
    sp.Matrix([0, d3/2, 0]),
    sp.zeros(3, 1)
]
masses = [0, m1, m2, m3]
inertias = [sp.zeros(3) for _ in range(4)]  # Point masses, so no inertia
joint_types = [0, 'R', 'R', 'P']

# Define gravity
gravity = sp.Matrix([0, 0, -g])

# Define symbols for the variables (for Lagrange)
variables = theta1, theta2, d3

# Solve the inverse dynamics, print the equations in latex
equations_lagrange = rtk.dynamics_lagrange(transforms, pos_coms, masses, inertias, joint_types, gravity, variables)
rtk.print_equations_dict(equations_lagrange, latex=True)

$$\tau_{0} = 0$$
$$\tau_{1} = \ddot{\theta_1} \left(\frac{d_{3}^{2} m_{2} \sin^{2}{\left(\theta_{2} \right)}}{4} + d_{3}^{2} m_{3} \sin^{2}{\left(\theta_{2} \right)}\right) + \frac{\dot{\theta_1} \dot{\theta_2} d_{3}^{2} m_{2} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)}}{2} + 2 \dot{\theta_1} \dot{\theta_2} d_{3}^{2} m_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)} + \frac{\dot{\theta_1} \dot{d_3} d_{3} m_{2} \sin^{2}{\left(\theta_{2} \right)}}{2} + 2 \dot{\theta_1} \dot{d_3} d_{3} m_{3} \sin^{2}{\left(\theta_{2} \right)}$$
$$\tau_{2} = \ddot{\theta_2} \left(\frac{d_{3}^{2} m_{2}}{4} + d_{3}^{2} m_{3}\right) - \frac{\dot{\theta_1}^{2} d_{3}^{2} m_{2} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)}}{4} - \dot{\theta_1}^{2} d_{3}^{2} m_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)} + \frac{\dot{\theta_2} \dot{d_3} d_{3} m_{2}}{2} + 2 \dot{\theta_2} \dot{d_3} d_{3} m_{3} - \frac{d_{3} g m_{2} \sin{\left(\theta_{2} \righ

Copy the $\LaTeX$ formatted equations into a markdown cell to print them all below.  
The individual kinetic and potential energies are included.

$$\tau_{0} = 0$$
$$\tau_{1} = \ddot{\theta_1} \left(\frac{d_{3}^{2} m_{2} \sin^{2}{\left(\theta_{2} \right)}}{4} + d_{3}^{2} m_{3} \sin^{2}{\left(\theta_{2} \right)}\right) + \frac{\dot{\theta_1} \dot{\theta_2} d_{3}^{2} m_{2} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)}}{2} + 2 \dot{\theta_1} \dot{\theta_2} d_{3}^{2} m_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)} + \frac{\dot{\theta_1} \dot{d_3} d_{3} m_{2} \sin^{2}{\left(\theta_{2} \right)}}{2} + 2 \dot{\theta_1} \dot{d_3} d_{3} m_{3} \sin^{2}{\left(\theta_{2} \right)}$$
$$\tau_{2} = \ddot{\theta_2} \left(\frac{d_{3}^{2} m_{2}}{4} + d_{3}^{2} m_{3}\right) - \frac{\dot{\theta_1}^{2} d_{3}^{2} m_{2} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)}}{4} - \dot{\theta_1}^{2} d_{3}^{2} m_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{2} \right)} + \frac{\dot{\theta_2} \dot{d_3} d_{3} m_{2}}{2} + 2 \dot{\theta_2} \dot{d_3} d_{3} m_{3} - \frac{d_{3} g m_{2} \sin{\left(\theta_{2} \right)}}{2} - d_{3} g m_{3} \sin{\left(\theta_{2} \right)}$$
$$\tau_{3} = \ddot{d_3} \left(\frac{m_{2}}{4} + m_{3}\right) - \frac{\dot{\theta_1}^{2} d_{3} m_{2} \sin^{2}{\left(\theta_{2} \right)}}{4} - \dot{\theta_1}^{2} d_{3} m_{3} \sin^{2}{\left(\theta_{2} \right)} - \frac{\dot{\theta_2}^{2} d_{3} m_{2}}{4} - \dot{\theta_2}^{2} d_{3} m_{3} + \frac{g m_{2} \cos{\left(\theta_{2} \right)}}{2} + g m_{3} \cos{\left(\theta_{2} \right)}$$
$$p_{c0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$p_{c1} = \left[\begin{matrix}0\\0\\\frac{L_{1}}{2}\end{matrix}\right]$$
$$p_{c2} = \left[\begin{matrix}- \frac{d_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{1} \right)}}{2}\\- \frac{d_{3} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} \right)}}{2}\\L_{1} + \frac{d_{3} \cos{\left(\theta_{2} \right)}}{2}\end{matrix}\right]$$
$$p_{c3} = \left[\begin{matrix}- d_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{1} \right)}\\- d_{3} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} \right)}\\L_{1} + d_{3} \cos{\left(\theta_{2} \right)}\end{matrix}\right]$$
$$v_{c0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$v_{c1} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$v_{c2} = \left[\begin{matrix}\frac{\dot{\theta_1} d_{3} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} \right)}}{2} - \frac{\dot{\theta_2} d_{3} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2} \right)}}{2} - \frac{\dot{d_3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{1} \right)}}{2}\\- \frac{\dot{\theta_1} d_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{1} \right)}}{2} - \frac{\dot{\theta_2} d_{3} \sin{\left(\theta_{1} \right)} \cos{\left(\theta_{2} \right)}}{2} - \frac{\dot{d_3} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} \right)}}{2}\\- \frac{\dot{\theta_2} d_{3} \sin{\left(\theta_{2} \right)}}{2} + \frac{\dot{d_3} \cos{\left(\theta_{2} \right)}}{2}\end{matrix}\right]$$
$$v_{c3} = \left[\begin{matrix}\dot{\theta_1} d_{3} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} \right)} - \dot{\theta_2} d_{3} \cos{\left(\theta_{1} \right)} \cos{\left(\theta_{2} \right)} - \dot{d_3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{1} \right)}\\- \dot{\theta_1} d_{3} \sin{\left(\theta_{2} \right)} \cos{\left(\theta_{1} \right)} - \dot{\theta_2} d_{3} \sin{\left(\theta_{1} \right)} \cos{\left(\theta_{2} \right)} - \dot{d_3} \sin{\left(\theta_{1} \right)} \sin{\left(\theta_{2} \right)}\\- \dot{\theta_2} d_{3} \sin{\left(\theta_{2} \right)} + \dot{d_3} \cos{\left(\theta_{2} \right)}\end{matrix}\right]$$
$$\omega_{0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$\omega_{1} = \left[\begin{matrix}0\\0\\\dot{\theta_1}\end{matrix}\right]$$
$$\omega_{2} = \left[\begin{matrix}\dot{\theta_1} \sin{\left(\theta_{2} \right)}\\\dot{\theta_1} \cos{\left(\theta_{2} \right)}\\\dot{\theta_2}\end{matrix}\right]$$
$$\omega_{3} = \left[\begin{matrix}\dot{\theta_1} \sin{\left(\theta_{2} \right)}\\- \dot{\theta_2}\\\dot{\theta_1} \cos{\left(\theta_{2} \right)}\end{matrix}\right]$$
$$K_{0} = 0$$
$$K_{1} = 0$$
$$K_{2} = \frac{m_{2} \left(\dot{\theta_1}^{2} d_{3}^{2} \sin^{2}{\left(\theta_{2} \right)} + \dot{\theta_2}^{2} d_{3}^{2} + \dot{d_3}^{2}\right)}{8}$$
$$K_{3} = \frac{m_{3} \left(\dot{\theta_1}^{2} d_{3}^{2} \sin^{2}{\left(\theta_{2} \right)} + \dot{\theta_2}^{2} d_{3}^{2} + \dot{d_3}^{2}\right)}{2}$$
$$V_{0} = 0$$
$$V_{1} = \frac{L_{1} g m_{1}}{2}$$
$$V_{2} = \frac{g m_{2} \cdot \left(2 L_{1} + d_{3} \cos{\left(\theta_{2} \right)}\right)}{2}$$
$$V_{3} = g m_{3} \left(L_{1} + d_{3} \cos{\left(\theta_{2} \right)}\right)$$
$$K_{total} = \left(\frac{m_{2}}{8} + \frac{m_{3}}{2}\right) \left(\dot{\theta_1}^{2} d_{3}^{2} \sin^{2}{\left(\theta_{2} \right)} + \dot{\theta_2}^{2} d_{3}^{2} + \dot{d_3}^{2}\right)$$
$$V_{total} = \frac{g \left(L_{1} m_{1} + m_{2} \cdot \left(2 L_{1} + d_{3} \cos{\left(\theta_{2} \right)}\right) + 2 m_{3} \left(L_{1} + d_{3} \cos{\left(\theta_{2} \right)}\right)\right)}{2}$$
$$L = - \frac{g \left(L_{1} m_{1} + m_{2} \cdot \left(2 L_{1} + d_{3} \cos{\left(\theta_{2} \right)}\right) + 2 m_{3} \left(L_{1} + d_{3} \cos{\left(\theta_{2} \right)}\right)\right)}{2} + \left(\frac{m_{2}}{8} + \frac{m_{3}}{2}\right) \left(\dot{\theta_1}^{2} d_{3}^{2} \sin^{2}{\left(\theta_{2} \right)} + \dot{\theta_2}^{2} d_{3}^{2} + \dot{d_3}^{2}\right)$$

Substitute all the necessary values for the Lagrangian:

In [22]:
lagrangian = equations_lagrange['L']
# Find what symbols need to be substituted
print(lagrangian.free_symbols)

{g, theta2, m1, \dot{d_3}, \dot{\theta_2}, d3, \dot{\theta_1}, L1, m3, m2}


In [23]:
# Define new variables for symbols that don't have any in this scope
# Required so we can substitute the values
theta_vel1, theta_vel2, d_vel3 = sp.symbols('\dot{\\theta_1}, \dot{\\theta_2}, \dot{d_3}')
subs_map = {
    L1:0.5,
    theta1:28 * sp.pi/180,
    theta2:79 * sp.pi/180,
    d3:0.225,
    theta_vel1:0.12,
    theta_vel2:0.05,
    d_vel3:0.56,
    g:10,
    m1:2.8,
    m2:1.4,
    m3:0.8
}
sp.N(lagrangian.subs(subs_map), 4)

-18.46

We can also find the numerical values of all of the derived quantities at once:

In [24]:
subbed_equations = rtk.substitute_equations_dict(equations_lagrange, subs_map)
print(rtk.free_symbols_equations_dict(subbed_equations))

{\ddot{d_3}, \ddot{\theta_1}, \ddot{\theta_2}}


In [25]:
numerical_equations = rtk.round_equations_dict(subbed_equations, 4)
rtk.print_equations_dict(numerical_equations, latex=True)

$$\tau_{0} = 0$$
$$\tau_{1} = 0.0561 \ddot{\theta_1} + 0.03364$$
$$\tau_{2} = 0.05822 \ddot{\theta_2} - 3.299$$
$$\tau_{3} = 1.15 \ddot{d_3} + 2.858$$
$$p_{c0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$p_{c1} = \left[\begin{matrix}0\\0\\0.25\end{matrix}\right]$$
$$p_{c2} = \left[\begin{matrix}-0.09751\\-0.05185\\0.5215\end{matrix}\right]$$
$$p_{c3} = \left[\begin{matrix}-0.195\\-0.1037\\0.5429\end{matrix}\right]$$
$$v_{c0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$v_{c1} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$v_{c2} = \left[\begin{matrix}-0.2374\\-0.1412\\0.0479\end{matrix}\right]$$
$$v_{c3} = \left[\begin{matrix}-0.4748\\-0.2825\\0.09581\end{matrix}\right]$$
$$\omega_{0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$\omega_{1} = \left[\begin{matrix}0\\0\\0.12\end{matrix}\right]$$
$$\omega_{2} = \left[\begin{matrix}0.1178\\0.0229\\0.05\end{matrix}\right]$$
$$\omega_{3} = \left[\begin{matrix}0.1178\\-0.05\\0.0229\end{matrix}\right]$$
$$K_{0} = 

$$\tau_{0} = 0$$
$$\tau_{1} = 0.0561 \ddot{\theta_1} + 0.03364$$
$$\tau_{2} = 0.05822 \ddot{\theta_2} - 3.299$$
$$\tau_{3} = 1.15 \ddot{d_3} + 2.858$$
$$p_{c0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$p_{c1} = \left[\begin{matrix}0\\0\\0.25\end{matrix}\right]$$
$$p_{c2} = \left[\begin{matrix}-0.09751\\-0.05185\\0.5215\end{matrix}\right]$$
$$p_{c3} = \left[\begin{matrix}-0.195\\-0.1037\\0.5429\end{matrix}\right]$$
$$v_{c0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$v_{c1} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$v_{c2} = \left[\begin{matrix}-0.2374\\-0.1412\\0.0479\end{matrix}\right]$$
$$v_{c3} = \left[\begin{matrix}-0.4748\\-0.2825\\0.09581\end{matrix}\right]$$
$$\omega_{0} = \left[\begin{matrix}0\\0\\0\end{matrix}\right]$$
$$\omega_{1} = \left[\begin{matrix}0\\0\\0.12\end{matrix}\right]$$
$$\omega_{2} = \left[\begin{matrix}0.1178\\0.0229\\0.05\end{matrix}\right]$$
$$\omega_{3} = \left[\begin{matrix}0.1178\\-0.05\\0.0229\end{matrix}\right]$$
$$K_{0} = 0$$
$$K_{1} = 0$$
$$K_{2} = 0.05503$$
$$K_{3} = 0.1258$$
$$V_{0} = 0$$
$$V_{1} = 7.0$$
$$V_{2} = 7.301$$
$$V_{3} = 4.343$$
$$K_{total} = 0.1808$$
$$V_{total} = 18.64$$
$$L = -18.46$$

### Newton-Euler Dynamics

> Consider a PR manipulator with the following transformation matrices
>$$
{}^0_1T = \left[\begin{matrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & d_1 \\
0 & 0 & 0 & 1
\end{matrix}\right], \quad
\
{}^1_2T = \left[\begin{matrix}
c_2 & -s_2 & 0 & 0 \\
0 & 0 & -1 & 0 \\
s_2 & c_2 & 0 & 0 \\
0 & 0 & 0 & 1
\end{matrix}\right], \quad
\
{}^2_3T = \left[\begin{matrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & L_2 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{matrix}\right]$$
> The mass centres of links 1 and 2 are located in the middle of the links. The length of link 1 is $L_1$. Both inertia tensors contain the principle inertia terms ($I_{xx}$, $I_{yy}$ and $I_{zz}$) only. The principal frame of each link is parallel to the DH frame and located at the mass centre. Acceleration due to gravity is given by
>$${}^0g = \left[\begin{matrix}-g\\0\\0\end{matrix}\right] = \left[\begin{matrix}-10\\0\\0\end{matrix}\right]$$
> An external force ${}^3f_3$ along the positive $x_3$ axis is applied at the tip of the manipulator.
>
> Find the analytical solution for the joint torques, and evaluate the solution with the following values:
> $$ d_1 = 1\ m, \quad \theta_2 = -\pi/6\ rad, \quad L_1 = 1\ m, \quad L_2 = 0.5\ m$$
> $$ \dot{d_1} = \dot{\theta_2} = 0, \quad \ddot{d_1} = 1\ m/s, \quad \ddot{\theta_2} = 1\ rad/s^2$$
> $$ m_1 = 10\ kg, \quad m_2 = 5\ kg$$
> $$ I_{xx1} = 0.05\ kgm^2, \quad I_{yy1} = 0.01\ kgm^2, \quad I_{zz1} = 0.2\ kgm^2$$
> $$ I_{xx2} = 0.05\ kgm^2, \quad I_{yy2} = 0.01\ kgm^2, \quad I_{zz2} = 0.1\ kgm^2$$
> $$ {}^3f_3 = 1\ N$$

Analytical solution:

In [26]:
# Define symbolic variables
d1, theta2 = sp.symbols('d1, theta2')
L1, L2, m1, m2, g = sp.symbols('L1, L2, m1, m2, g')
I_xx1, I_yy1, I_zz1, I_xx2, I_yy2, I_zz2 = sp.symbols('I_xx1, I_yy1, I_zz1, I_xx2, I_yy2, I_zz2')

# Define the manipulator
transforms = [
    rtk.trans(0, 0, d1),
    sp.Matrix([
        [sp.cos(theta2), -sp.sin(theta2), 0, 0],
        [0, 0, -1, 0],
        [sp.sin(theta2), sp.cos(theta2), 0, 0],
        [0, 0, 0, 1]
    ]),
    rtk.trans(0, L2, 0)
]
pos_coms = [
    sp.zeros(3,1),
    sp.Matrix([0, 0, -L1/2]),
    sp.Matrix([0, L2/2, 0])
]
masses = [0, m1, m2]
inertias = [
    sp.zeros(3),
    sp.Matrix([
        [I_xx1, 0, 0],
        [0, I_yy1, 0],
        [0, 0, I_zz1]
    ]),
    sp.Matrix([
        [I_xx2, 0, 0],
        [0, I_yy2, 0],
        [0, 0, I_zz2]
    ]),
]
joint_types = [0, 'P', 'R']

# Define gravity
gravity = sp.Matrix([-g, 0, 0])

# Define the boundary conditions (for Newton-Euler)
f_end = sp.Symbol('f_end')
f_end_effector = sp.Matrix([f_end, 0, 0])
n_end_effector = sp.zeros(3, 1)

# Solve the inverse dynamics
equations_newton_euler = rtk.dynamics_newton_euler(transforms, pos_coms, masses, inertias, joint_types, gravity, f_end_effector, n_end_effector)

In [27]:
equations_newton_euler.keys()

dict_keys(['tau', 'omega', 'alpha', 'a', 'a_c', 'f_c', 'n_c', 'f', 'n'])

In [28]:
# Get just the joint torque equations
equations_joint_torques = {'tau': equations_newton_euler['tau']}
rtk.print_equations_dict(equations_joint_torques, latex=True)

$$\tau_{0} = 0$$
$$\tau_{1} = - \frac{L_{2} \ddot{\theta_2} m_{2} \sin{\left(\theta_{2} \right)}}{2} - \frac{L_{2} \dot{\theta_2}^{2} m_{2} \cos{\left(\theta_{2} \right)}}{2} + \ddot{d_1} \left(m_{1} + m_{2}\right) + f_{end} \sin{\left(\theta_{2} \right)}$$
$$\tau_{2} = - \frac{L_{2} \ddot{d_1} m_{2} \sin{\left(\theta_{2} \right)}}{2} - L_{2} f_{end} - \frac{L_{2} g m_{2} \cos{\left(\theta_{2} \right)}}{2} + \ddot{\theta_2} \left(I_{zz2} + \frac{L_{2}^{2} m_{2}}{4}\right)$$


$$\tau_{0} = 0$$
$$\tau_{1} = - \frac{L_{2} \ddot{\theta_2} m_{2} \sin{\left(\theta_{2} \right)}}{2} - \frac{L_{2} \dot{\theta_2}^{2} m_{2} \cos{\left(\theta_{2} \right)}}{2} + \ddot{d_1} \left(m_{1} + m_{2}\right) + f_{end} \sin{\left(\theta_{2} \right)}$$
$$\tau_{2} = - \frac{L_{2} \ddot{d_1} m_{2} \sin{\left(\theta_{2} \right)}}{2} - L_{2} f_{end} - \frac{L_{2} g m_{2} \cos{\left(\theta_{2} \right)}}{2} + \ddot{\theta_2} \left(I_{zz2} + \frac{L_{2}^{2} m_{2}}{4}\right)$$

Evaluate with the given values:

In [29]:
print(rtk.free_symbols_equations_dict(equations_joint_torques))

{g, theta2, I_zz2, \ddot{d_1}, L2, m1, \dot{\theta_2}, m2, \ddot{\theta_2}, f_end}


In [30]:
d_vel1, theta_vel2, d_accel1, theta_accel2 = sp.symbols('\dot{d_1}, \dot{\\theta_2}, \ddot{d_1}, \ddot{\\theta_2}')
subs_map = {
    d1:1,
    theta2:-sp.pi/6,
    L1:1,
    L2:0.5,
    d_vel1:0,
    theta_vel2:0,
    d_accel1:1,
    theta_accel2:1,
    m1:10,
    m2:5,
    I_xx1:0.05,
    I_yy1:0.01,
    I_zz1:0.2,
    I_xx2:0.05,
    I_yy2:0.01,
    I_zz2:0.1,
    f_end:1,
    g:10
}
subbed_equations = rtk.substitute_equations_dict(equations_joint_torques, subs_map)
numerical_equations = rtk.round_equations_dict(subbed_equations, 4)
rtk.print_equations_dict(numerical_equations, latex=True)

$$\tau_{0} = 0$$
$$\tau_{1} = 15.13$$
$$\tau_{2} = -10.29$$


$$\tau_{0} = 0$$
$$\tau_{1} = 15.13$$
$$\tau_{2} = -10.29$$